In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Simulation Group Project

/content/drive/.shortcut-targets-by-id/1O-neExZo97YeLSIQySxWveY7zDguU94N/Simulation Group Project


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
from multiprocessing import Pool, cpu_count
import functools

# Parse pathway features

In [ ]:
entrez2symbol = {}
with open('input/Homo_sapiens.gene_info') as f:
    for line in f.read().splitlines():
        row = line.split('\t')
        entrez2symbol[row[1]]=row[2]

In [ ]:
gene2pathway = {}
pathways = set()
with open('NBSS-master/data/hallmarks.txt') as f:
    for line in f.read().splitlines():
        row = line.split('\t')
        if len(row) > 1:
            pathway = row[0].split('|')[1]
            pathways.add(pathway)
            for entrez in row[2:]:
                if entrez not in entrez2symbol:
                    continue
                gene = entrez2symbol[entrez]
                if gene not in gene2pathway:
                    gene2pathway[gene]=set()
                gene2pathway[gene].add(pathway)

# Parse TCGA LUAD subtypes

In [ ]:
df = pd.read_csv("input/LUAD_query_subtype.csv")
new_df = df[['patient', 'iCluster.Group']].rename(columns={'patient': 'Sample', 'iCluster.Group': 'Group'})

In [ ]:
import os

folder_name = '/content/drive/My Drive/Simulation Group Project/test'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

file_path = os.path.join(folder_name, 'TCGA_LUAD_subtypes.csv')
new_df.to_csv(file_path, sep='\t', index=False)

In [ ]:
pat2subtype = {}
def parse_subtypes(fn, pat2subtype):
    with open(fn) as f:
        for line in f.read().rstrip().splitlines()[1:]:
            row = line.split("\t")
            pat = row[0][:12]
            if len(row[0])>12:
                tissue_code = row[0][13:15]
                if int(tissue_code) >= 10:
                    continue
            #Remove "Normal" subtype
            if row[1] in ['NA', 'Normal']:
                continue
            pat2subtype[pat] = row[1]
        return pat2subtype

pat2subtype = parse_subtypes(file_path, pat2subtype)

# Parse mutation and CNA profile

In [ ]:
def parse_maf(fn, oncogene_tsg):
    df = pd.read_table(fn, low_memory=False)
    df = df.loc[(df.loc[:,'is_flank']==0) & (df.loc[:,'is_silent']==0),:]
    df['pat'] = df.loc[:,'Tumor_Sample_Barcode'].str[:12]

    filter_rows = []
    genes = set()
    for index, row in df.iterrows():
        gene = row['Hugo_Symbol']
        # (Optional) filter cancer genes
        # if gene not in cancergenes:
        #     continue
        genes.add(gene)
        VC = row['Variant_Classification']
        if gene in oncogene_tsg:
            if oncogene_tsg[gene] in ['Oncogene']:
                if VC not in ['Missense_Mutation', 'In_Frame_Del', 'In_Frame_Ins', 'De_novo_Start_InFrame']:
                    filter_rows.append(index)
            if oncogene_tsg[gene] == 'Amplification_Oncogene':
                filter_rows.append(index)
    df = df.drop(filter_rows)

    df = df.loc[:,['pat','Hugo_Symbol']]
    df['counter'] = 1
    df.set_index(['pat','Hugo_Symbol'], inplace=True)
    df = df.counter.groupby(level=[0,1]).min().unstack()
    df.fillna(0, inplace=True)
    return df, genes

In [ ]:
oncogene_tsg={}
with open('NBSS-master/data/oncogene_tsg.txt') as f:
    for line in f.read().rstrip().splitlines():
        row = line.split("\t")
        oncogene_tsg[row[0]] = row[1]

In [ ]:
fn = 'input/LUAD-TP.maf.annotated'
with open(fn, encoding="utf8", errors='ignore') as f:
  df_mut, genes = parse_maf(f, oncogene_tsg)

In [ ]:
def parse_CNA(fn, genes):
    df = pd.read_table(fn,low_memory=False,index_col=0)
    df = df[df.index.isin(genes)]
    # # (Optional) filter cancer genes
    # df = df[df.index.isin(cancergenes)]
    df = df.iloc[:,2:]
    df = (df/2.).round(0)
    df.columns = df.columns.str[:12]

    nonOncogene_rows = []
    nonOGTSG_rows = []
    for index, row in df.iterrows():
        gene = index
        if not (gene in oncogene_tsg and oncogene_tsg[gene] in ['Oncogene', 'Amplification_Oncogene']):
            nonOncogene_rows.append(index)
        if gene not in oncogene_tsg:
            nonOGTSG_rows.append(index)
    df.loc[nonOncogene_rows,:] = df.loc[nonOncogene_rows,:] * (-1)
    df.loc[nonOGTSG_rows,:] = df.loc[nonOGTSG_rows,:] * (0)
    df = df.clip(lower=0)
    return df

In [ ]:
coding_genes = set()
with open("input/ref-transcripts.gtf") as f:
    next(f)  # Skip the first line
    for line in f:
        row = line.rstrip().split("\t")
        if len(row) < 9:  # Ensure there are enough elements in the row
            continue
        if row[2] != "transcript":  # Check for 'transcript' feature
            continue
        attributes = row[8]
        gene_name = None
        # Split attributes by semicolon and strip extra whitespace
        for attribute in attributes.split(';'):
            attribute = attribute.strip()
            if attribute.startswith('gene '):
                gene_name = attribute.split(' ')[1].replace('"', '')  # Remove the double quotes
                break
        if gene_name:
            coding_genes.add(gene_name)

In [ ]:
coding_genes = coding_genes | genes

In [ ]:
fn = 'input/all_thresholded.by_genes.txt'
df_CNA = parse_CNA(fn, coding_genes)

In [ ]:
df_mut_CNA = pd.concat([df_mut, df_CNA.transpose()], axis=1,
                       join='inner').transpose().groupby(level=0).sum().clip(upper=1.).transpose()
df_mut_CNA = df_mut_CNA.loc[set(df_mut.index) & set(df_CNA.columns) & set(pat2subtype.keys()),df_CNA.index]

<ipython-input-16-b6289ac8bf9e>:3: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_mut_CNA = df_mut_CNA.loc[set(df_mut.index) & set(df_CNA.columns) & set(pat2subtype.keys()),df_CNA.index]


In [ ]:
df_mut_CNA = df_mut_CNA[(df_mut_CNA.T != 0).any()] # remove samples with all zeros
df_mut_CNA.shape

(220, 20916)

# Seperating training, validation and testing samples

In [ ]:
training_set = df_mut_CNA.sample(frac=2./3)
training_set.sort_index(inplace=True)

In [ ]:
validation_set = df_mut_CNA.drop(training_set.index).sample(frac=1./2)
validation_set.sort_index(inplace=True)

In [ ]:
testing_set = df_mut_CNA.drop(training_set.index).drop(validation_set.index)
testing_set.sort_index(inplace=True)

In [ ]:
recurrently_mutated_genes = training_set.loc[:,training_set.sum()>=8].columns
training_set = training_set.loc[:,recurrently_mutated_genes]
validation_set = validation_set.loc[:,recurrently_mutated_genes]
testing_set = testing_set.loc[:,recurrently_mutated_genes]
training_set = training_set[(training_set.T != 0).any()]
validation_set = validation_set[(validation_set.T != 0).any()]
testing_set = testing_set[(testing_set.T != 0).any()]


In [ ]:
training_set.shape

(147, 748)

# Parse PathwayCommons

In [ ]:
PathwayCommons = pd.read_table('input/PathwayCommons9.All.hgnc.txt')

<ipython-input-23-467bb32c2922>:1: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  PathwayCommons = pd.read_table('input/PathwayCommons9.All.hgnc.txt')


In [ ]:
PathwayCommons = PathwayCommons.loc[PathwayCommons.loc[:,'INTERACTION_TYPE'].isin(['controls-state-change-of',
                                                                                   'controls-transport-of',
                                                                                   'controls-phosphorylation-of',
                                                                                   'controls-expression-of',
                                                                                   'catalysis-precedes',
                                                                                   'in-complex-with',
                                                                                   'interacts-with',
                                                                                   'neighbor-of']),:]

In [ ]:
def parse_edge_features(mutrates, df):
    edge2features = {}
    for index, row in df.iterrows():
        g1 = row['PARTICIPANT_A']
        g2 = row['PARTICIPANT_B']

        # (Optionally) filter by cancer genes or pathways
        # if not (g1 in cancergenes and g2 in cancergenes):
        #     continue

        #(Optionally) filter by frequently mutated genes
        if not (g1 in recurrently_mutated_genes and g2 in recurrently_mutated_genes):
            continue

        ty = row['INTERACTION_TYPE']
        ty_d = ty + '_d'
        ty_rev = ty + '_rev'
        sources = row['INTERACTION_DATA_SOURCE'].split(';')
        edge = g1 + '\t' + g2
        edge_rev = g2 + '\t' + g1
        if edge not in edge2features:
            edge2features[edge] = {'gene 1':g1, 'gene 2':g2}
        if edge_rev not in edge2features:
            edge2features[edge_rev] = {'gene 1':g2, 'gene 2':g1}

        # Parse edge type features
        if ty not in ['in-complex-with','interacts-with','neighbor-of']:
            edge2features[edge][ty_d] = 1.
            edge2features[edge_rev][ty_rev] = 1.
        else:
            edge2features[edge][ty] = 1.
            edge2features[edge_rev][ty] = 1.

        # Parse edge source features
        for source in sources:
            edge2features[edge][source] = 1.
            edge2features[edge_rev][source] = 1.

        # Parse pathway features.
        # If one node is in the pathway, the score is 0.5
        # If both nodes are in the pathway, the score is 1
        if g1 in gene2pathway or g2 in gene2pathway:
            for pathway in pathways:
                edge2features[edge][pathway] = 0.
                edge2features[edge_rev][pathway] = 0.
            for g in [g1, g2]:
                if g in gene2pathway:
                    for pathway in gene2pathway[g]:
                        edge2features[edge][pathway] += 0.5
                        edge2features[edge_rev][pathway] += 0.5

        # Parse mutation features from the training set
        # Calculate mutation rates
        mutrate_g1 = 0
        mutrate_g2 = 0
        if g1 in training_set:
            mutrate_g1 = mutrates.loc[g1]
        if g2 in training_set:
            mutrate_g2 = mutrates.loc[g2]
        edge2features[edge]['mutrate_source'] = mutrate_g1
        edge2features[edge]['mutrate_target'] = mutrate_g2
        edge2features[edge_rev]['mutrate_source'] = mutrate_g2
        edge2features[edge_rev]['mutrate_target'] = mutrate_g1

        # (optional) add fisher test against subtypes
#         if g1 in gene2fisherp:
#             edge2features[edge]['fisherp_source'] = gene2fisherp[g1]
#             edge2features[edge_rev]['fisherp_target'] = gene2fisherp[g1]
# #             for subtype in gene2fisherp[g1]:
# #                 edge2features[edge]['{}_source'.format(subtype)] = gene2fisherp[g1][subtype]
# #                 edge2features[edge_rev]['{}_target'.format(subtype)] = gene2fisherp[g1][subtype]
#         if g2 in gene2fisherp:
#             edge2features[edge]['fisherp_target'] = gene2fisherp[g2]
#             edge2features[edge_rev]['fisherp_source'] = gene2fisherp[g2]
# #             for subtype in gene2fisherp[g2]:
# #                 edge2features[edge]['{}_target'.format(subtype)] = gene2fisherp[g2][subtype]
# #                 edge2features[edge_rev]['{}_source'.format(subtype)] = gene2fisherp[g2][subtype]

        # Calculate mutual exclusivity / co-occurrence
        ME = 0.
        if g1 in training_set and g2 in training_set:
            if training_set.loc[:,g1].sum() < 8 or training_set.loc[:,g2].sum() < 8:
                continue
            tab = pd.crosstab(training_set.loc[:,g1],training_set.loc[:,g2])
            if tab.shape != (2, 2):
                continue
            if tab.iloc[1,1] * tab.iloc[0,0] >= tab.iloc[0,1] * tab.iloc[1,0]:
                continue
            oddsratio, pvalue = fisher_exact(tab, alternative='less')
            logp = np.log10(pvalue)
            ME = -logp
            if pvalue < 0.05:
                print(g1, g2, oddsratio, pvalue)
        edge2features[edge]['mutual_exclusive'] = ME
        edge2features[edge_rev]['mutual_exclusive'] = ME

    return edge2features

In [ ]:
training_set_mutrate = training_set.sum() / training_set.shape[0]

n_processes = cpu_count()
pool = Pool(processes=n_processes)

df_split = np.array_split(PathwayCommons, n_processes, axis=0)
parse_edge_features_partial = functools.partial(parse_edge_features, training_set_mutrate)
edge2features_list = pool.map(parse_edge_features_partial, df_split)

pool.close()
pool.join()

KRAS PIK3CA 0.13354037267080746 0.01972708900333445
KRAS PIK3CA 0.13354037267080746 0.01972708900333445
PRKCB KRAS 0.0 0.02764645536970553
PRKCB KRAS 0.0 0.02764645536970553
EGFR KRAS 0.07707509881422925 0.0008328909117289691
SETD2 TP53 0.19130434782608696 0.0209634868779738
STK11 TP53 0.2864583333333333 0.005087288196089028
STK11 TP53 0.2864583333333333 0.005087288196089028
STK11 TP53 0.2864583333333333 0.005087288196089028
STK11 TP53 0.2864583333333333 0.005087288196089028


In [ ]:
import collections

def update(d, u):
    for k, v in u.iteritems():
        if isinstance(v, collections.Mapping):
            r = update(d.get(k, {}), v)
            d[k] = r
        else:
            d[k] = u[k]
    return d

In [ ]:
edge2features_list

[{'A2M\tADAM19': {'gene 1': 'A2M',
   'gene 2': 'ADAM19',
   'interacts-with': 1.0,
   'HPRD': 1.0,
   'TGF-beta signaling pathway': 0.0,
   'Focal adhesion': 0.0,
   'Jak-STAT signaling pathway': 0.0,
   'Adherens junction': 0.0,
   'Epithelial-mesenchymal transition': 0.0,
   'Pathways in cancer': 0.0,
   'p53 signaling pathway': 0.0,
   'inflammatory response': 0.5,
   'VEGF signaling pathway': 0.0,
   'PPAR signaling pathway': 0.0,
   'ErbB signaling pathway': 0.0,
   'mTOR signaling pathway': 0.0,
   'telomere maintenance': 0.0,
   'T cell receptor signaling pathway': 0.0,
   'Apoptosis': 0.0,
   'Wnt signaling pathway': 0.0,
   'Natural killer cell mediated cytotoxicity': 0.0,
   'ECM-receptor interaction': 0.0,
   'Glycolysis / Gluconeogenesis': 0.0,
   'Cell cycle': 0.0,
   'Nucleotide excision repair': 0.0,
   'ESC proliferation': 0.0,
   'Mismatch repair': 0.0,
   'Cytokine-cytokine receptor interaction': 0.0,
   'B cell receptor signaling pathway': 0.0,
   'Base excision rep

In [ ]:
edge2features = {}
for edge_feature in edge2features_list:
    edge2features.update(edge_feature)


In [ ]:
edge2features_df = pd.DataFrame.from_dict(edge2features, orient='index')

In [ ]:
edge2features_df = edge2features_df.fillna(0).sort_index(1)
cols = list(edge2features_df)
cols.insert(0, cols.pop(cols.index('gene 2')))
cols.insert(0, cols.pop(cols.index('gene 1')))
edge2features_df = edge2features_df[cols]

<ipython-input-31-f96cc7067b5a>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  edge2features_df = edge2features_df.fillna(0).sort_index(1)


# Output dataframes to files

In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/1O-neExZo97YeLSIQySxWveY7zDguU94N/Simulation Group Project


In [ ]:
training_set.to_csv('test/LUAD_training_data_2.txt', sep='\t')
validation_set.to_csv('test/LUAD_validation_data_2.txt', sep='\t')
testing_set.to_csv('test/LUAD_testing_data_2.txt', sep='\t')

In [ ]:
training_set.index.to_series().map(pat2subtype).to_csv('test/LUAD_training_lables_2.txt', sep='\t')
validation_set.index.to_series().map(pat2subtype).to_csv('test/LUAD_validation_lables_2.txt', sep='\t')
testing_set.index.to_series().map(pat2subtype).to_csv('test/LUAD_testing_lables_2.txt', sep='\t')

In [ ]:
edge2features_df.to_csv('test/LUAD_edge2features_2.txt', sep='\t', header=False, index=False)

In [ ]:
with open('test/LUAD_feature_names_2.txt', 'w') as f:
    f.write('\n'.join(edge2features_df.columns[2:]))

# Scratch

In [ ]:
edge2features_df.loc['PIK3CA\tTP53',:]

gene 1                                 PIK3CA
gene 2                                   TP53
Adherens junction                         0.0
Apoptosis                                 1.0
B cell receptor signaling pathway         0.5
                                       ...   
mutrate_target                       0.482993
mutual_exclusive                          0.0
p53 signaling pathway                     0.5
pid                                       0.0
telomere maintenance                      0.0
Name: PIK3CA\tTP53, Length: 64, dtype: object

In [ ]:
training_set.sum().sort_values()[::-1].head(10)

Gene Symbol
TTN      72.0
TP53     71.0
MUC16    59.0
RYR2     56.0
CSMD3    56.0
USH2A    51.0
KRAS     47.0
LRP1B    46.0
ZFHX4    43.0
FLG      41.0
dtype: float64

In [ ]:
edge2features_df.iloc[:,2:].sum().sort_values()[::-1]

in-complex-with               1934.0
Reactome                      1826.0
PANTHER                       1422.0
interacts-with                1282.0
Focal adhesion                1093.0
                               ...  
WikiPathways                    10.0
KEGG                             8.0
Base excision repair             0.0
Mismatch repair                  0.0
Nucleotide excision repair       0.0
Length: 62, dtype: float64

In [ ]:
edge2features_df

,gene 1,gene 2,Adherens junction,Apoptosis,B cell receptor signaling pathway,BIND,Base excision repair,BioGRID,CORUM,CTD,...,in-complex-with,inflammatory response,interacts-with,mTOR signaling pathway,mutrate_source,mutrate_target,mutual_exclusive,p53 signaling pathway,pid,telomere maintenance
A2M\tADAM19,A2M,ADAM19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,1.0,0.0,0.068027,0.061224,0.283459,0.0,0.0,0.0
ADAM19\tA2M,ADAM19,A2M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,1.0,0.0,0.061224,0.068027,0.283459,0.0,0.0,0.0
A2M\tADAMTS12,A2M,ADAMTS12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,1.0,0.0,0.068027,0.204082,0.000000,0.0,0.0,0.0
ADAMTS12\tA2M,ADAMTS12,A2M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,1.0,0.0,0.204082,0.068027,0.000000,0.0,0.0,0.0
A2M\tADAMTS1,A2M,ADAMTS1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.5,1.0,0.0,0.068027,0.054422,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCDHGA2\tZIC1,PCDHGA2,ZIC1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.061224,0.068027,0.000000,0.0,0.0,0.0
ZIC1\tSALL1,ZIC1,SALL1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.068027,0.068027,0.000000,0.0,0.0,0.0
SALL1\tZIC1,SALL1,ZIC1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.068027,0.068027,0.000000,0.0,0.0,0.0
ZIC1\tSLIT3,ZIC1,SLIT3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.068027,0.074830,0.000000,0.0,0.0,0.0


In [ ]:
recurrently_mutated_genes

Index(['MTOR', 'PTCHD2', 'VPS13D', 'TAS1R2', 'UBR4', 'HSPG2', 'ARID1A',
       'CSMD2', 'DLGAP3', 'MACF1',
       ...
       'MAGEC1', 'SLITRK4', 'SLITRK2', 'AFF2', 'GPR50', 'PASD1', 'CNGA2',
       'GABRQ', 'L1CAM', 'F8'],
      dtype='object', name='Gene Symbol', length=748)